# RAG Tutorial - From Basic do Advanced

The goal of this tutorial is to explore a set of techniques for giving LLMs context over private, recent or specific data in order to avoid LLMs hallucinations or them not being able to give a proper answer.

This set of techniques are commonly known as RAG, that stands for Retrieval Augmented Generation.
<br>
<br>

_**Warning**_ <br>
Before diving into the world of RAG, we strongly recommend you, mainly if you are not very familiarized with the field, to read the glossary below. It will give you a basic understanding of fundamental topics regarding Generative AI, Machine Learning and LLMs, crutial in order to better understand RAG.

## Glossary

#### Machine Learning

#### Generative AI

#### Large Language Models (LLMs)

#### Query

#### Hallucinations

#### Embeddings

#### Vector Database

#### Semantic Search

#### System Message

#### Chunking

## Tech Stack

The following libraries and technologies will be used in the development of this tutorial and the applications of RAG within it.

* Langchain/Llama Index
* Pinecone/MongoDB - Vector Database
* LLMs APIs (OpenAI, Anthropic, Claude)...

## What is RAG and how it works?

RAG is a technique for giving LLMs context over private, recent or specific data that the large language model had no previous access to. Its goal is to avoid LLMs hallucinations or them not being able to give a proper answer.

The process of creating an infrastructure for implementing RAG consists on 4 steps
* Getting the data (specific context you want the LLM to know)
* Chunking the data (dividing it into small pieces - chunks)
* Embedding the chunks (transforming the chunks into 'lots of numbers' - vectors of dimension _n_)
* Storing the embeddings in a vector database

When this infrastructure is built, the prompting workflow works as below:

(IMAGE SHOWING BASIC RAG WORKFLOW)

As shown, the process has 3 main stages:
* Embedding
* Retrieval
* Generation

That workflow is showing the most simple process a RAG application undertakes, in which the user prompt (query) is embedded and then a similairty search is conducted in order to find the chunks that are more related to the prompt made. After that, the most related chunks (top-k chunks) are added to the user prompt and fed into the LLM. The LLM now have (hopefully) not just the user prompt but the necessary context to answer it properly. 

# Example Application - Doing xyzxyzxyzxyz
Now we will conduct an application of the RAG technique, starting with a simple RAG application and exploring more advanced approaches later on. The goal of this example is to explore the different variables of RAG infrastructure and how they might affect the quality of the answer provided by the LLM.

Different (i) embedding models, (ii) vector databases, (iii) similarity searches and (iv) retrieval techniques will be explored.

At the end of the day, our main goal when applying RAG it to increase the assertiviness and quality of the document that is retrieved as context, in order to give the LLM enough information to provide a proper answer.


obs pra mim: Testar pelo menos duas formas de embeddar, duas bases de dados (pensar se esse aq é válido), duas formas de calcular similaridade, duas formas de retrieve contexto.

## Application Context

## Pre-Configuration - Setting API Keys

In [1]:
from dotenv import load_dotenv

import os

load_dotenv('secrets.env')

True

In [2]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_API_ENV = os.getenv("PINECONE_API_ENV")

## Loading Documents

In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
# PDF Manual do Aluno Insper
loader = PyPDFLoader('files/manualdoaluno.pdf')

In [5]:
manual = loader.load()

In [6]:
manual[1]

Document(page_content=' \n \n \nInsper  Instituto  de Ensino  e Pesquisa  \nPortaria MEC nº270, 13/02/2020 – D.O.U. 17/02/2020  \n \n2 \n \n \nPublicação: 01/ 2024                                                          Rua Quatá, 300 – Vila Olímpia 04546 -042 São Paulo SP Brasil  \n   55 11 4504 -2400 www.insper.edu.br  \n                                                                             \n \nSUMÁRIO  \n1. BOAS -VINDAS  ................................ ................................ ................................ ................................ ................ 4 \n2. SOBRE OS CURSOS  ................................ ................................ ................................ ................................ ......5 \n2.1. MODELO DE FORMAÇÃO INTEGRADA  ................................ ................................ ............................... 5 \n2.2. APRENDIZADO CENTRADO NO ALUNO  ................................ ................................ ..........

## Chunking Documents

In [7]:
# Criando objeto que criará chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, # 1000 caracteres
    chunk_overlap  = 100, # caracteres em comum entre o chunks subsequentes - garante continuidade e contexto
    length_function = len, # como estamos usando chunk_size em termos de caracteres, usaremos len como função para medir tamanho
    # Obs: Poderíamos trocar a função len por uma que conte 1 para cada palavra, e determinar chunk size em termos de palavras
)

In [8]:
# Criando os chunks
texts = text_splitter.split_documents(manual)
texts

[Document(page_content='Rua Quatá, 300 – Vila Olímpia 04546 -042 São Paulo SP Brasil  \n55 11 4504 -2400 www.insper.edu.br  \n \n Insper  Instituto  de Ensino  e Pesquisa  \nPortaria MEC nº270, 13/02/2020 – D.O.U. 17/02/2020  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nGraduação | Administração                                                          \ne Ciências Econômicas  \nManual do Aluno  \n2024 \n \n \n \n \n \n \nÁrea responsável:  Secretaria Acadêmica de Graduação  \n                                                                 Data de p ublicação:  janeiro /2024', metadata={'source': 'files/manualdoaluno.pdf', 'page': 0}),
 Document(page_content='Insper  Instituto  de Ensino  e Pesquisa  \nPortaria MEC nº270, 13/02/2020 – D.O.U. 17/02/2020  \n \n2 \n \n \nPublicação: 01/ 2024                                                          Rua Quatá, 300 – Vila Olímpia 04546 -042 São Paulo SP Brasil  \n   55 11 4504 -2400 www.insper.edu.br  \n         

In [9]:
# Contabilizando chunks
n_chunks = len(texts)
print(f'Número de Chunks de Texto é de {n_chunks}')

Número de Chunks de Texto é de 106


In [10]:
# Avaliando o número de tokens por chunk
# Essa avaliação nos dará o tamanho do chunk em termos de token
# Assim podemos evitar extrapolar a janela de contexto do LLM
import tiktoken

encoding = tiktoken.encoding_for_model('gpt-3.5-turbo')

# Exemplo de chunk
text_chunk = texts[2].page_content

# Encodind o chunk em tokens
tokens = encoding.encode(text_chunk)

# Contando o número de tokens
num_tokens = len(tokens)

print(f"Número de tokens: {num_tokens}")

Número de tokens: 172


In [11]:
# Criando uma lista com todos os 'page_content' dos chunks
manual_contents = [chunk.page_content for chunk in manual]
manual_contents

[' \n \n \n \n                                                                                        Rua Quatá, 300 – Vila Olímpia 04546 -042 São Paulo SP Brasil  \n55 11 4504 -2400 www.insper.edu.br  \n \n Insper  Instituto  de Ensino  e Pesquisa  \nPortaria MEC nº270, 13/02/2020 – D.O.U. 17/02/2020  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nGraduação | Administração                                                          \ne Ciências Econômicas  \nManual do Aluno  \n2024 \n \n \n \n \n \n \nÁrea responsável:  Secretaria Acadêmica de Graduação  \n                                                                 Data de p ublicação:  janeiro /2024  \n  ',
 ' \n \n \nInsper  Instituto  de Ensino  e Pesquisa  \nPortaria MEC nº270, 13/02/2020 – D.O.U. 17/02/2020  \n \n2 \n \n \nPublicação: 01/ 2024                                                          Rua Quatá, 300 – Vila Olímpia 04546 -042 São Paulo SP Brasil  \n   55 11 4504 -2400 www.insper.edu.br  

## Embedding Chunks and Creating vector Database

In [12]:
# Importando dependências necessárias
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings # Classe para chamada de Modelo de Embedding
import pinecone # Biblioteca do Vector Database

In [13]:
embeddings_model = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY,
    model='text-embedding-ada-002' # Modelo de embedding -> opções: 'text-embedding-3-small', 'text-embedding-3-large'
)

/home/joaohcs/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [14]:
pinecone_client = pinecone.Pinecone(
                                    api_key=PINECONE_API_KEY,
                                    environment=PINECONE_API_ENV
                                    )

index_name= 'test-manual'

In [15]:
# Rode na primeira vez para criar o vector DB na sua conta do pinecone
# Da segunda vez, pode rodar apenas o código abaixo para puxar o index já criado
manual_search = Pinecone.from_texts(
    texts=manual_contents,
    embedding=embeddings_model,
    index_name=index_name
)

In [16]:
# manual_search = pinecone.Index(index_name, host = 'https://test-manual-n26yis3.svc.aped-4627-b74a.pinecone.io')

In [17]:
## Testando uma query e vendo resultados similares
query = 'Quais os trade-offs entre estudar economia e administração?'
retrieved_docs = manual_search.similarity_search(query)
retrieved_docs

[Document(page_content=' \n \n \nInsper  Instituto  de Ensino  e Pesquisa  \nPortaria MEC nº270, 13/02/2020 – D.O.U. 17/02/2020  \n \n6 \n \n \nPublicação: 01/ 2024                                                          Rua Quatá, 300 – Vila Olímpia 04546 -042 São Paulo SP Brasil  \n   55 11 4504 -2400 www.insper.edu.br  \n                                                                             \n \n▪ manter um ambiente de mútuo respeito e favorável à troca de ideias, para que cada aluno \nse sinta seguro a expor seu ponto de vista.  \n2.3. Competências que pretendemos desenvolver nos alunos  \nToda a plataforma de ensino e aprendizagem do Insper (currículo, princípios e práticas pedagógicas, \natividades extracurriculares e recursos de apoio acadêmico)  visa desenvolver nos alunos os \nseguintes conhecimentos e competências, denominados objetivos de aprendizagem da graduação : \na. Conhecimento específico em Administração e Economia, conforme o curso \nescolhido.  \nEntender com

## Query Augmentation (Adding most similar Context to User Prompt)

In [18]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [19]:
llm = OpenAI(temperature=0.4, api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type='stuff')

/home/joaohcs/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [20]:
# Resposta do LLM
print(chain.run(input_documents=retrieved_docs, question=query))

/home/joaohcs/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



Não há informações suficientes no contexto fornecido para responder a essa pergunta. O texto menciona as competências e conhecimentos específicos desenvolvidos em ambos os cursos, mas não discute os trade-offs entre eles. Além disso, o texto não compara diretamente os dois cursos, apenas menciona que é possível obter dupla titulação entre eles. Portanto, não é possível responder a essa pergunta com base no contexto fornecido.


# Implementando ChatBot

In [21]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

In [31]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    model='gpt-3.5-turbo',
    temperature=0.4
)

In [32]:
messages = [
    SystemMessage(content='Você é um bot que ajuda calouros da Faculdade Insper com dúvidas acerca da graduação.'),
    HumanMessage(content='Olá, eu sou um aluno do Insper. Gostaria de saber mais sobre a faculdade e suas regras.'),
    AIMessage(content='Ótimo! Estou aqui para te ajudar com qualquer pergunta acerca do seu curso e faculdade.'),
    HumanMessage(content='Quais os princípios fundamentais que devem nortear o dia a dia dos membros da comunidade insper?')
]

In [24]:
# Exemplo de Resposta do Chat
resp = chat(messages)
print(resp.content)

/home/joaohcs/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Os princípios fundamentais que norteiam o dia a dia dos membros da comunidade Insper são:

1. Excelência acadêmica e profissional: Buscar sempre a excelência em tudo o que se faz, seja no âmbito acadêmico, profissional ou pessoal.

2. Ética e integridade: Agir com ética e integridade em todas as situações, respeitando sempre os valores morais e normas da instituição.

3. Colaboração e respeito: Valorizar a colaboração, o trabalho em equipe e respeitar a diversidade de opiniões e de pessoas.

4. Empreendedorismo e inovação: Estimular o espírito empreendedor e a busca por soluções inovadoras para os desafios do mundo.

5. Responsabilidade socioambiental: Promover ações e atitudes responsáveis com o meio ambiente e com a sociedade.

Estes princípios guiam a comunidade Insper em direção a uma formação integral e de qualidade.


In [25]:
# Adicionando resposta do chat ao histórico
messages.append(resp)

prompt_novo = HumanMessage(
    content='Tem certeza?'

)

# Adicionando o prompt novo do user
messages.append(prompt_novo)

resp2 = chat(messages)

In [26]:
print(resp2.content)

Peço desculpas pela confusão anterior. Aqui estão os princípios fundamentais que devem nortear o dia a dia dos membros da comunidade Insper:

1. Excelência: Buscar sempre a excelência em todas as atividades acadêmicas e profissionais.

2. Ética e Integridade: Agir com ética e integridade em todas as situações, respeitando os valores morais e normas da instituição.

3. Respeito e Diversidade: Valorizar o respeito mútuo, a diversidade de opiniões e experiências, e promover um ambiente inclusivo.

4. Colaboração: Estimular a colaboração e o trabalho em equipe entre os membros da comunidade.

5. Espírito Empreendedor: Incentivar a criatividade, a inovação e o empreendedorismo na busca por soluções para os desafios.

6. Responsabilidade Social e Ambiental: Promover a consciência e a responsabilidade socioambiental, contribuindo para um mundo mais sustentável.

Estes são os princípios fundamentais que orientam a comunidade Insper em suas atividades diárias. Espero ter esclarecido sua dúvida.

## Adicionando contexto com RAG

In [33]:
def augment_prompt(query: str):
    ret_docs = manual_search.similarity_search(query, k=3)
    
    source_knowledge = "\n".join([i.page_content for i in ret_docs])

    augmented_prompt = f"""Usando o contexto abaixo, responda a pergunta query. Se a pergunta não estiver relacionada ao contexto, esqueça o contexto dado e responda como você normalmente responderia, pois você ainda é um assistente disposto a ajudar o aluno Insper. E se apesar da pergunta não estiver relacionada você não saber a resposta, apenas diga que não sabe como normalmente você faria.

    Contextos:
    {source_knowledge}

    Query: {query}"""
    return augmented_prompt

In [28]:
prompt = HumanMessage(
    content=augment_prompt('Quais são os principios que norteiam o dia a dia do aluno insper? Tente de novo.')
)

# adicionando às mensagens
messages.append(prompt)

# Resposta do chat
resp = chat(messages)

print(resp.content)

Os princípios que norteiam o dia a dia do aluno Insper são:

1. Manter um ambiente de mútuo respeito e favorável à troca de ideias, para que cada aluno se sinta seguro a expor seu ponto de vista.

2. Buscar a excelência acadêmica e profissional em todas as atividades realizadas.

3. Agir com ética e integridade em todas as situações, respeitando os valores morais e normas da instituição.

4. Colaborar e respeitar a diversidade de opiniões e de pessoas.

5. Estimular o empreendedorismo, a inovação e a responsabilidade socioambiental.

Esses princípios são fundamentais para a formação integral dos alunos e para a construção de uma comunidade acadêmica engajada e comprometida com a excelência.


## Criando Chat Local pra teste

In [34]:
import gradio as gr

def predict(message, history):
    history_langchain_format = [SystemMessage(content='Você é um assistente que ajuda alunos Insper, você é muito do bem e gente boa. Responda sempre na língua na qual dor perguntado, pois você também ajuda estudantes internacionais. Busque responder as perguntas para ajudar os alunos Insper quando conseguir!')]
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=augment_prompt(message)))
    gpt_response = chat(history_langchain_format)
    return gpt_response.content

In [35]:
gr.ChatInterface(predict).launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


# Example application 2 - (Exploring Different Advanced Technique)